In [1]:
%cd ../..

/home/eli/AnacondaProjects/combinators


In [2]:
import logging

import probtorch
import torch

import combinators
import combinators.model as model
import combinators.model.foldable as foldable
from combinators.inference import importance, mcmc
import combinators.utils as utils

import examples.gmm.gmm as gmm
import examples.hmm.hmm as hmm

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    key = 'Pi_%d' % k if k else 'Pi'
    hmm_params[key] = {'concentration': torch.ones(5)}

In [4]:
init_gmm = gmm.InitGmm(params=hmm_params, trainable=False, batch_shape=(1,))
init_hmm = hmm.InitHmm(params=hmm_params, trainable=False, batch_shape=(1,))
init_hmm = model.Compose(init_hmm, init_gmm)
hmm_foldable = foldable.Step(hmm.HmmStep(), initializer=init_hmm)
hmm_run = foldable.Reduce(hmm_foldable, generator=lambda: range(50))

In [ ]:
(z_last, mu, sigma, pi), generative, _ = hmm_run()

In [ ]:
data = {k: v.value.squeeze() for k, v in generative.variables() if 'X_' in k}

In [ ]:
num_particles = 250

In [ ]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    key = 'Pi_%d' % k if k else 'Pi'
    smc_hmm_params[key] = {'concentration': torch.ones(5)}

In [ ]:
init_gmm = gmm.InitGmm(params=smc_hmm_params, trainable=True, batch_shape=(num_particles,))
init_hmm = hmm.InitHmm(params=smc_hmm_params, trainable=True, batch_shape=(num_particles,))
init_hmm = model.Compose(init_hmm, init_gmm)

In [ ]:
hmm_step = foldable.Step(hmm.HmmStep(batch_shape=(num_particles,)), initializer=init_hmm)
hmm_reduce = foldable.Reduce(hmm_step, lambda: range(50))

In [ ]:
rmsmc_hmm = mcmc.step_resample_move_smc(hmm_reduce, moves=50)

In [ ]:
samples, inference, _ = rmsmc_hmm(data=data)

In [ ]:
latent_states = {k: v.value for k, v in generative.variables() if 'Z_' in k}

In [ ]:
inferred_latent_states = {k: v.value for k, v in inference.variables() if 'Z_' in k}

In [ ]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inferred_latent_states[key] == latent_states[key]).to(dtype=torch.float).mean()
    print('RM-SMC percent accuracy at time %d: %f' % (t, accuracy * 100))